In [1]:
# import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd 

In [2]:
# Set executable path and initialize the chrome browser in splinter 
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path)

# Web Scrapping NASA

In [3]:
# visit NASA website 
url= 'https://mars.nasa.gov/news/'
browser.visit(url)
#Optional delay for website 
# Here we are searching for elements with a specific combination of tag (ul) and (li) and attriobute (item_lit) and (slide)
# Ex. being <ul class= "item_list">
browser.is_element_not_present_by_css("ul.item_list li.slide", wait_time=1)

False

In [4]:
# HTML Parser
html= browser.html 
news_soup= soup(html, 'html.parser')
#slide_elem looks for <ul /> tags and descendents <li />
# the period(.) is used for selecting classes such as item_list
slide_elem= news_soup.select_one('ul.item_list li.slide')

In [5]:
# Chained the (.find) to slide_elem which says this var holds lots of info, so look inside to find this specific entity
slide_elem.find('div', class_= 'content_title')

<div class="content_title"><a href="/news/8756/my-culture-my-voice/" target="_self">My Culture, My Voice</a></div>

### Get Title

In [6]:
# Need just the title, not Html extras
news_title= slide_elem.find("div", class_= 'content_title').get_text()
news_title

'My Culture, My Voice'

In [7]:
# Get article body
news_p= slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'In honor of Hispanic Heritage Month, Christina Hernandez, an instrument engineer on the Mars 2020 mission, talks about her childhood and journey to NASA.'

### Featured Images

In [8]:
# Visit URL 
url= 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mar'
browser.visit(url)

In [9]:
# Find and click the full_image button
full_image_elem= browser.find_by_id('full_image')
full_image_elem.click()

In [10]:
# Find the more info button and click that 
# is_element_present_by_text() method to search for an element that has the provided text
browser.is_element_present_by_text('more info', wait_time=1)

# will take our string 'more info' and add link associated with it, then click
more_info_elem=browser.links.find_by_partial_text('more info')
more_info_elem.click()

In [20]:
# Parse the resulting html with soup
html=browser.html
img_soup=soup(html, 'html.parser')

In [21]:
# Find the relative image url 

# The 'figure.lede' references the <figure /> tag and its class=lede
# the 'a' is the next tag nested inside the <figure /> tag, as well as the 'img' tag 
# the .get('src') pulls the link to the image


# WE are telling soup to go to figure tag, then within that look for an 'a' tag then within that look foir a 'img' tag
img_url_rel= img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA18049_hires.jpg'

In [22]:
# Need to get the FULL URL 
img_url= f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18049_hires.jpg'

### Get Facts about Mars

In [24]:
# Creating DF by telling function to look for first html table in site it encounters by indexing it to zero
df=pd.read_html('http://space-facts.com/mars/')[0]

# Assigning columns 
df.columns=['description', 'value']
# Creating description column as the index 
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [25]:
# WE can convert it back to HTML table 
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [26]:
# end the site 
browser.quit()